In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score ,  roc_curve, auc , classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
import xgboost as xgb 
import statistics
from scipy.fftpack import fft, ifft

!pip install biopython
import Bio

from itertools import product

import collections

from imblearn.over_sampling import SMOTE 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.6 MB/s eta 0:00:00


In [2]:
# Record Constants
INPUT_TRAIN_IN = "train_in_small.csv"
INPUT_TRAIN_OUT = "train_out_small.csv"  
INPUT_TEST_IN = "test_in.csv"
INPUT_TEST_OUT = "test_out.csv"

WINDOW_SIZE =  100 # Final RNA Sequence will be 201 Length
MAX_LENGTH = (WINDOW_SIZE * 2)+1

In [3]:
#Read X Varaibles and Y Varaibles

x_train_raw =  pd.read_csv( INPUT_TRAIN_IN, header=None , skiprows=1 )
y_train_raw =  pd.read_csv(  INPUT_TRAIN_OUT, header=None , skiprows=1 )

x_test_raw =  pd.read_csv( INPUT_TEST_IN, header=None , skiprows=1)
y_test_raw =  pd.read_csv( INPUT_TEST_OUT, header=None , skiprows=1)

In [5]:
def headerNAC(k):
    caracteres = ['A', 'C', 'G', 'T']
    listTotal = []
    permsList = [''.join(str(i) for i in x) for x in product(caracteres, repeat=k)]
    # print ('permsList')
    # print (permsList)
    for perm in permsList:
        # print (perm)
        listTotal.append(perm)
    return listTotal

In [58]:
def chunks_two(seq, win):
    # print('seq in chunks_two')
    # print(seq)
    # print('win')
    # print(win)
    seqlen = len(seq)
    for i in range(seqlen):
        j = seqlen if i+win>seqlen else i+win
        yield seq[i:j]
        if j==seqlen: break
    return

In [74]:
def nacSeq(seq):
    global probabilities
    global listTotal
    listTotal = headerNAC(3)
    caracteres = ['A', 'C', 'G', 'T']
    for seq_record in seq:
        # print('seq in nacSeq')
        # print(seq)
        # print('seq_record in nacSeq')
        # print(seq_record)
        probabilities = []
        kmer = {}
        totalWindows = (200 - 3) + 1  # (L - k + 1)
        permsList = [''.join(str(i) for i in x) for x in product(caracteres, repeat=3)]
        for key in permsList:
            kmer[key] = 0
        kmer = collections.OrderedDict(sorted(kmer.items()))
        # print('kmer')
        # print(kmer)
        for subseq in chunks_two(seq_record, 3):
            if subseq in kmer:
                # print('subseq in kmer')
                # print(subseq)
                kmer[subseq] = kmer[subseq] + 1
            else:
                kmer[subseq] = 1
        for key, value in kmer.items():
            # print('key')
            # print (key)
            # print ('value')
            # print (value)
            probabilities.append([str(key), value/totalWindows])
    return probabilities

In [75]:
def getExactPos(x_train_raw):
    middle_index = (x_train_raw.shape[1] // 2) + 1
    truncated_df = x_train_raw.iloc[:, middle_index - WINDOW_SIZE - 1: middle_index + WINDOW_SIZE]
    return truncated_df 

x_train_split = getExactPos(x_train_raw)
x_train_split

,400,401,402,403,404,405,406,407,408,409,...,591,592,593,594,595,596,597,598,599,600
0,G,G,G,C,G,C,C,T,C,G,...,C,C,T,C,T,C,C,T,A,G
1,A,A,C,A,G,T,A,A,A,C,...,T,T,T,T,T,T,T,T,T,C
2,A,A,A,G,A,A,C,A,T,G,...,C,A,G,A,A,T,T,A,G,G
3,G,G,G,T,G,G,T,G,C,C,...,G,T,C,T,C,T,A,A,G,A
4,T,C,A,T,T,G,A,A,G,T,...,A,A,A,T,A,T,G,C,T,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,A,T,T,T,G,A,T,G,T,T,...,T,G,T,A,T,A,A,T,A,N
1409,G,A,G,T,T,G,A,G,T,G,...,G,T,T,A,T,G,A,G,A,N
1410,C,T,C,A,C,A,G,G,A,A,...,T,C,T,G,T,G,C,T,A,N
1411,G,T,T,T,T,T,A,A,A,C,...,T,C,G,G,C,T,T,A,N,N


In [77]:
# x_train_split.to_csv("out.csv")

x_train_combined =  pd.read_csv( "out.csv", header=None , skiprows=1 )

x_train_combined

,0
0,GGGCGCCTCGGCTCTTGTCCACGCCCTGGACCGCGGGGACCGCGAG...
1,AACAGTAAACCAAATGCATTGTAACATCTTATTTAATTCTAACATG...
2,AAAGAACATGCTTCTTCCTTCTTAGTATATAGATCAGGATCCACTT...
3,GGGTGGTGCCTGGACAGAGTGAGTGTCTAATGATTGCTAAAGTAAT...
4,TCATTGAAGTTTTTTTGATCACAGGAATAACATGTTTAGATAGTAA...
...,...
1408,ATTTGATGTTCACCTTTTATACCATAATCCTGTTCATTGAAAAACA...
1409,GAGTTGAGTGAATCCAGGTGAGTACGGATGTTGCTCCTGCCCCTTC...
1410,CTCACAGGAAATGGTTCCCAAGGCTATGCCTGAGATAAGGCTGGCT...
1411,GTTTTTAAACTTTTCTAAAATATTACTTACCAAAATATTTTCCTCT...


In [78]:
result = x_train_combined.apply(nacSeq, axis=1)

In [79]:
result

0       [[AAA, 0.0], [AAC, 0.0], [AAG, 0.0202020202020...
1       [[AAA, 0.050505050505050504], [AAC, 0.02525252...
2       [[AAA, 0.010101010101010102], [AAC, 0.01515151...
3       [[AAA, 0.025252525252525252], [AAC, 0.01515151...
4       [[AAA, 0.04040404040404041], [AAC, 0.015151515...
                              ...                        
1408    [[AAA, 0.030303030303030304], [AAC, 0.02020202...
1409    [[AAA, 0.03535353535353535], [AAC, 0.030303030...
1410    [[AAA, 0.005050505050505051], [AAC, 0.01010101...
1411    [[AAA, 0.03535353535353535], [AAC, 0.005050505...
1412    [[AAA, 0.025252525252525252], [AAC, 0.00505050...
Length: 1413, dtype: object

In [80]:
result.to_csv("output.csv")
result

0       [[AAA, 0.0], [AAC, 0.0], [AAG, 0.0202020202020...
1       [[AAA, 0.050505050505050504], [AAC, 0.02525252...
2       [[AAA, 0.010101010101010102], [AAC, 0.01515151...
3       [[AAA, 0.025252525252525252], [AAC, 0.01515151...
4       [[AAA, 0.04040404040404041], [AAC, 0.015151515...
                              ...                        
1408    [[AAA, 0.030303030303030304], [AAC, 0.02020202...
1409    [[AAA, 0.03535353535353535], [AAC, 0.030303030...
1410    [[AAA, 0.005050505050505051], [AAC, 0.01010101...
1411    [[AAA, 0.03535353535353535], [AAC, 0.005050505...
1412    [[AAA, 0.025252525252525252], [AAC, 0.00505050...
Length: 1413, dtype: object

In [ ]:
y_train_raw

In [ ]:
for rownum,(indx,val) in enumerate(result.iteritems()):
    print('row number: ', rownum, 'index: ', indx, 'value: ', val)

In [130]:
# Import writer class from csv module
from csv import writer
 
# List that we want to add as a new row
List = [6, 'William', 5532, 1, 'UAE']
 

data = pd.DataFrame()
index = 0

for rownum,(indx,val) in enumerate(result.iteritems()):
  oneRow = []
  for j in val:
    oneRow.append(j[1])
  # print('rownum')
  # print(len(oneRow))
  # if(len(oneRow) != 64):
  #   oneRow = []
  # data[rownum] = oneRow
  # df = pd.DataFrame(oneRow)
  # print(df)
  # frames = [data, pd.DataFrame(oneRow)]
  # data = pd.concat(frames)
  # index +=1
  
  # Open our existing CSV file in append mode
  # Create a file object for this file
  with open('features.csv', 'a') as f_object:
  
      # Pass this file object to csv.writer()
      # and get a writer object
      writer_object = writer(f_object)
  
      # Pass the list as an argument into
      # the writerow()
      writer_object.writerow(oneRow)
  
      # Close the file object
      f_object.close()

data

Streaming output truncated to the last 5000 lines.

[64 rows x 1 columns]
           0
0   0.000000
1   0.005051
2   0.010101
3   0.000000
4   0.005051
..       ...
59  0.030303
60  0.000000
61  0.025253
62  0.010101
63  0.015152

[64 rows x 1 columns]
           0
0   0.000000
1   0.010101
2   0.010101
3   0.000000
4   0.010101
..       ...
59  0.020202
60  0.005051
61  0.015152
62  0.000000
63  0.005051

[64 rows x 1 columns]
           0
0   0.025253
1   0.005051
2   0.005051
3   0.025253
4   0.010101
..       ...
59  0.025253
60  0.035354
61  0.015152
62  0.035354
63  0.121212

[64 rows x 1 columns]
           0
0   0.055556
1   0.035354
2   0.040404
3   0.030303
4   0.020202
..       ...
59  0.010101
60  0.020202
61  0.015152
62  0.015152
63  0.025253

[64 rows x 1 columns]
           0
0   0.020202
1   0.005051
2   0.025253
3   0.020202
4   0.015152
..       ...
59  0.020202
60  0.020202
61  0.035354
62  0.015152
63  0.025253

[64 rows x 1 columns]
           0
0   0.015152
1   0

,0
0,0.000000
1,0.000000
2,0.020202
3,0.005051
4,0.005051
...,...
60,0.015152
61,0.015152
62,0.010101
63,0.030303


In [135]:
features =  pd.read_csv( "features.csv", header=None , skiprows=0 )
features

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.000000,0.000000,0.020202,0.005051,0.005051,0.050505,0.015152,0.005051,0.015152,0.000000,...,0.010101,0.020202,0.000000,0.020202,0.015152,0.005051,0.000000,0.005051,0.010101,0.000000
1,0.050505,0.025253,0.005051,0.045455,0.020202,0.010101,0.000000,0.010101,0.005051,0.030303,...,0.005051,0.030303,0.030303,0.010101,0.015152,0.015152,0.025253,0.030303,0.015152,0.070707
2,0.010101,0.015152,0.005051,0.020202,0.020202,0.005051,0.000000,0.015152,0.020202,0.000000,...,0.005051,0.040404,0.020202,0.040404,0.005051,0.015152,0.030303,0.050505,0.030303,0.055556
3,0.025253,0.015152,0.030303,0.045455,0.020202,0.010101,0.000000,0.015152,0.035354,0.015152,...,0.000000,0.030303,0.030303,0.010101,0.015152,0.015152,0.035354,0.015152,0.010101,0.050505
4,0.040404,0.015152,0.010101,0.035354,0.025253,0.005051,0.000000,0.020202,0.010101,0.005051,...,0.000000,0.025253,0.020202,0.010101,0.025253,0.020202,0.030303,0.015152,0.030303,0.070707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,0.030303,0.020202,0.015152,0.040404,0.030303,0.020202,0.000000,0.005051,0.020202,0.015152,...,0.000000,0.005051,0.030303,0.000000,0.015152,0.030303,0.025253,0.015152,0.040404,0.050505
1409,0.035354,0.030303,0.015152,0.020202,0.010101,0.000000,0.005051,0.030303,0.025253,0.005051,...,0.005051,0.010101,0.030303,0.020202,0.015152,0.020202,0.025253,0.025253,0.020202,0.035354
1410,0.005051,0.010101,0.020202,0.015152,0.005051,0.015152,0.005051,0.025253,0.020202,0.005051,...,0.000000,0.035354,0.015152,0.025253,0.035354,0.025253,0.000000,0.030303,0.015152,0.020202
1411,0.035354,0.005051,0.010101,0.030303,0.015152,0.005051,0.005051,0.015152,0.010101,0.005051,...,0.005051,0.035354,0.005051,0.015152,0.010101,0.015152,0.050505,0.030303,0.010101,0.101010


In [141]:
clf = OneVsRestClassifier(xgb.XGBClassifier(n_jobs=-1, max_depth=4 , objective='multi:softmax' , num_class = 12))
clf.fit(features , y_train_raw )

OneVsRestClassifier(estimator=XGBClassifier(max_depth=4, n_jobs=-1,
                                            num_class=12,
                                            objective='multi:softmax'))

In [ ]:
y_test_f = clf.predict(x_test_raw)

# Calculate the accuracy score of the xgboost regression model
accuracy = accuracy_score(y_test_raw, y_test_f)

In [146]:
print("Accuracy of XGBoost  model: {:.2f}".format(accuracy))

Accuracy of XGBoost  model: 0.53
